In [3]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from skimage.feature import local_binary_pattern
from skimage.feature import graycomatrix, graycoprops
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob

# Download Images



In [2]:
#Downlaod images from drive/MyDrive/CS419/Outex_TC_00012

"""
source_dir = '/content/drive/MyDrive/CS419/Assignment4/Outex_TC_00012'
images_dir = '/content/drive/MyDrive/CS419/Assignment4/Outex_TC_00012/images'

"""

images_dir = './imageset/images/'


train_0 = []
test_0 = []
test_1 = []

filenames = os.listdir(images_dir)

#sort filenames to correctly match the labels
filenames.sort()
#print(filenames)

#read images
for i in range(480):
  image = cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE)
  train_0.append(image)

for i in range(480, 4800):
  test_0.append(cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE))

for i in range(4800, len(filenames)):
  test_1.append(cv2.imread(os.path.join(images_dir, filenames[i]), cv2.IMREAD_GRAYSCALE))

print(len(train_0), len(test_0), len(test_1))

480 4320 4320


In [3]:
#converting lists to arrays and checking the shape

train_0 = np.array(train_0)
test_0 = np.array(test_0)
test_1 = np.array(test_1)
print(train_0.shape, test_0.shape, test_1.shape)

(480, 128, 128) (4320, 128, 128) (4320, 128, 128)


# Feature Extractors


### 1.Local Binary Patterns (LBP)

In [4]:
def compute_lbp(image, radius=2, n_points=16, method='uniform'):
  """
  Compute Local Binary Pattern (LBP) of an image.

  Args:
    image: Grayscale input image.
    radius: Radius of the circular neighborhood.
    n_points: Number of points in the neighborhood.
    method: Method to compute LBP ('uniform' or 'default').

  Returns:
    LBP histogram (feature vector).
  """
  lbp = local_binary_pattern(image, n_points, radius, method)
  # Histogram of LBP
  n_bins = int(lbp.max() + 1)
  hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
  hist = hist / hist.sum()  # Normalize histogram
  hist = np.array(hist)
  return hist

### 2. GLCM (Gray-Level Co-occurrence Matrix)

In [5]:
# Compute GLCM and extract features
def compute_glcm_features(gray_image, distances, angles):
  glcm = graycomatrix(
    gray_image,
    distances=distances,
    angles=angles,
    levels=256,  # 8-bit image has 256 levels
    symmetric=True,
    normed=True
  )

  # Extract features
  contrast = graycoprops(glcm, 'contrast')
  dissimilarity = graycoprops(glcm, 'dissimilarity')
  homogeneity = graycoprops(glcm, 'homogeneity')
  energy = graycoprops(glcm, 'energy')
  correlation = graycoprops(glcm, 'correlation')

  features = {
    'Contrast': contrast,
    'Correlation': correlation,
    'Energy': energy,
    'Homogeneity': homogeneity,
    'Dissimilarity': dissimilarity
  }

  feature_vector = []
  #how to initialize an empyt numpy array


  for key, value in features.items():
    for v in value:
      for k in v:
        feature_vector.append(k)

  feature_vector = np.array(feature_vector)

  return glcm, feature_vector


distances = [1, 2, 3]  # Pixel distances
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Angles in radians

print(compute_glcm_features(train_0[0], distances, angles)[1])


[1.29297244e+02 2.00602393e+02 1.19780758e+02 2.51701593e+02
 3.52921193e+02 2.00602393e+02 3.33189608e+02 2.51701593e+02
 5.03768250e+02 4.72101600e+02 4.62945312e+02 5.53210066e+02
 8.38477600e-01 7.49415698e-01 8.50319813e-01 6.85574989e-01
 5.60106520e-01 7.49415698e-01 5.84182054e-01 6.85574989e-01
 3.72317450e-01 4.12329148e-01 4.23398444e-01 3.11352291e-01
 2.16116038e-02 1.94366186e-02 2.18652359e-02 1.87320424e-02
 1.77923841e-02 1.94366186e-02 1.78587575e-02 1.87320424e-02
 1.69425730e-02 1.69218481e-02 1.69893694e-02 1.66313788e-02
 1.02688617e-01 7.98704550e-02 1.10413555e-01 7.68038696e-02
 6.63750650e-02 7.98704550e-02 6.86084034e-02 7.68038696e-02
 5.57158519e-02 5.45557389e-02 5.84547817e-02 5.19947098e-02
 9.05351870e+00 1.14906070e+01 8.69525098e+00 1.27577035e+01
 1.49519469e+01 1.14906070e+01 1.45932540e+01 1.27577035e+01
 1.78196250e+01 1.75738851e+01 1.72329375e+01 1.89007937e+01]


### 3. FFT Features of the Texture Image

In [6]:
def compute_fft_features(image, num_rings=5, num_wedges=8):
  # Compute FFT and shift zero frequency to center
  fft = np.fft.fft2(image)
  fft_shifted = np.fft.fftshift(fft)
  magnitude_spectrum = np.abs(fft_shifted)

  # Get image dimensions and center
  h, w = image.shape
  cy, cx = h // 2, w // 2
  max_radius = int(np.sqrt(cy**2 + cx**2))

  # Create radius and angle maps
  y, x = np.ogrid[:h, :w]
  radius = np.sqrt((y - cy)**2 + (x - cx)**2)
  angle = np.arctan2(y - cy, x - cx) % (2 * np.pi)

  # Compute ring features
  ring_edges = np.linspace(0, max_radius, num_rings + 1)
  ring_features = [
    magnitude_spectrum[(radius >= ring_edges[i]) & (radius < ring_edges[i + 1])].sum()
    for i in range(num_rings)
  ]

  # Compute wedge features
  wedge_edges = np.linspace(0, 2 * np.pi, num_wedges + 1)
  wedge_features = [
    magnitude_spectrum[(angle >= wedge_edges[i]) & (angle < wedge_edges[i + 1])].sum()
    for i in range(num_wedges)
  ]

  # Flatten features by concatenating ring and wedge features
  flattened_features = np.array(ring_features + wedge_features)

  return flattened_features

image = train_0[0]

# Extract FFT features
num_rings = 5  # Number of rings
num_wedges = 8  # Number of wedges
fft_features = compute_fft_features(image, num_rings, num_wedges)

# Visualize features
print("FFT Features:", fft_features)


FFT Features: [7528569.57085799 7764039.4722744  3294343.46178634 1309487.60913622
  155390.19115493 4318971.59726571 1880077.63356567 2656243.32990535
 2206772.5047341  2233002.00015262 1890703.22069978 2670094.91703946
 2195994.10184718]


### 4. Gabor Features

In [7]:
def extract_gabor_features(image, ksize=25, sigma=5.0, lambd=5.0, gamma=0.5, psi=0, orientations=None):
    """
    Extract flattened Gabor features from an image using multiple orientations.

    Args:
        image_path (str): Path to the grayscale image.
        ksize (int): Size of the Gabor filter kernel.
        sigma (float): Standard deviation of the Gaussian envelope.
        lambd (float): Wavelength of the sinusoidal factor.
        gamma (float): Aspect ratio.
        psi (float): Phase offset.
        orientations (list): List of orientations in radians. Defaults to [0, π/4, π/2, 3π/4].

    Returns:
        flattened_features (np.ndarray): Flattened feature vector (mean and std for each orientation).
        filtered_images (list): List of filtered images for each orientation.
        kernels (list): List of Gabor filter kernels for each orientation.
    """
    if orientations is None:
        orientations = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4]  # Default orientations

    # Lists to store results
    filtered_images = []
    kernels = []
    gabor_features = []

    # Apply Gabor filters for each orientation
    for theta in orientations:
        # Create Gabor kernel
        kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        mean = filtered.mean()
        std = filtered.std()
        gabor_features.append(mean)
        gabor_features.append(std)
        filtered_images.append(filtered)
        kernels.append(kernel)

    gabor_features = np.array(gabor_features)

    return gabor_features, filtered_images, kernel


# Create Feature Vectors

In [31]:
def create_feature_vectors(train_0 : np.ndarray, test_0 : np.ndarray , test_1 : np.ndarray, distances=[1, 2, 3, 4, 5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """
    Create feature vectors for training and test sets.

    Args:
        train_0 (array): List of training images (class 0).
        test_0 (array): List of test images (class 0).
        test_1 (array): List of test images (class 1).

    Returns:
        train (dict): Dictionary containing feature vectors for training set.
        test_0_dict (dict): Dictionary containing feature vectors for test set 0.
        test_1_dict (dict): Dictionary containing feature vectors for test set 1.

    """


    # Initialize dictionaries to store feature vectors for training and test sets
    train = {}
    test_0_dict = {}
    test_1_dict = {}

    # Initialize lists to store feature vectors
    lbp = []
    glcm = []
    fft = []
    gabor = []

    # Compute features for training images
    for image in train_0:
        lbp.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm.append(image_features)  # Compute GLCM features
        fft.append(compute_fft_features(image))  # Compute FFT features
        flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
        gabor.append(flattened_features)  # Compute Gabor features

    # Store computed features in the training dictionary
    train["LBP"] = np.array(lbp)
    train["GLCM"] = np.array(glcm)
    train["FFT"] = np.array(fft)
    train["Gabor"] = np.array(gabor)

    # Reset feature lists for test_0
    lbp0 = []
    glcm0 = []
    fft0 = []
    gabor0 = []

    # Compute features for test_0 images
    for i, image in enumerate(test_0):
        lbp0.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm0.append(image_features)  # Compute GLCM features
        fft0.append(compute_fft_features(image))  # Compute FFT features
        flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
        gabor0.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_0 dictionary
    test_0_dict["LBP"] = np.array(lbp0)
    test_0_dict["GLCM"] = np.array(glcm0)
    test_0_dict["FFT"] = np.array(fft0)
    test_0_dict["Gabor"] = np.array(gabor0)

    # Reset feature lists for test_1
    lbp1 = []
    glcm1 = []
    fft1 = []
    gabor1 = []

    # Compute features for test_1 images
    for image in test_1:
        lbp1.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm1.append(image_features)  # Compute GLCM features
        fft1.append(compute_fft_features(image))  # Compute FFT features
        flattened_features, _, _ = extract_gabor_features(image)  # Compute Gabor features
        gabor1.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_1 dictionary
    test_1_dict["LBP"] = np.array(lbp1)
    test_1_dict["GLCM"] = np.array(glcm1)
    test_1_dict["FFT"] = np.array(fft1)
    test_1_dict["Gabor"] = np.array(gabor1)

    print("\nShapes:")
    print(f"Training set: {train['LBP'].shape}, {train['GLCM'].shape}, {train['FFT'].shape}, {train['Gabor'].shape}")
    print(f"Test set 0: {test_0_dict['LBP'].shape}, {test_0_dict['GLCM'].shape}, {test_0_dict['FFT'].shape}, {test_0_dict['Gabor'].shape}")
    print(f"Test set 1: {test_1_dict['LBP'].shape}, {test_1_dict['GLCM'].shape}, {test_1_dict['FFT'].shape}, {test_1_dict['Gabor'].shape}")


    return train, test_0_dict, test_1_dict

In [13]:
initial_train, initial_test_0, initial_test_1 = create_feature_vectors(train_0, test_0, test_1)


Shapes:
Training set: (480, 18), (480, 60), (480, 13), (480, 8)
Test set 0: (4320, 18), (4320, 60), (4320, 13), (4320, 8)
Test set 1: (4320, 18), (4320, 60), (4320, 13), (4320, 8)


# Distance Functions

In [14]:
def euclidean_distance(x, y):
    """
    Compute the Euclidean distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).

    Returns:
        Euclidean distance (float).
    """
    return np.sqrt(np.sum((x - y) ** 2))


def manhattan_distance(x, y):
    """
    Compute the Manhattan distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).

    Returns:
        Manhattan distance (float).
    """
    return np.sum(np.abs(x - y))


def chi_squared_distance(x, y, epsilon=1e-10):
    """
    Compute the Chi-squared distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).
        epsilon: Small constant to avoid division by zero (default: 1e-10).

    Returns:
        Chi-squared distance (float).
    """
    return np.sum((x - y) ** 2 / (x + y + epsilon))


def compute_inverse_covariance_matrix(train_features):
    """
    Compute the regularized inverse covariance matrix.

    Args:
        train_features (np.ndarray): Feature vectors of training data.

    Returns:
        inv_cov_matrix (np.ndarray): Regularized inverse covariance matrix.
    """
    # Compute the covariance matrix
    cov_matrix = np.cov(train_features, rowvar=False)
    
    # Add a small regularization term to the diagonal
    regularization_term = 1e-5  # Adjust this value if needed
    cov_matrix += np.eye(cov_matrix.shape[0]) * regularization_term
    
    # Compute the inverse
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    return inv_cov_matrix



def mahalanobis_distance(x, y, inv_cov_matrix):
    """
    Compute the Mahalanobis distance between two feature vectors.

    Args:
        x: Feature vector 1 (1D array).
        y: Feature vector 2 (1D array).
        inv_cov_matrix: Inverse covariance matrix of the training data.

    Returns:
        Mahalanobis distance (float).
    """
    diff = x - y
    return np.sqrt(np.dot(np.dot(diff.T, inv_cov_matrix), diff))






# Nearest Neighbour Algorithm

In [15]:
def nearest_neighbour(train_feature_vector : np.ndarray, train_labels : np.ndarray, test_feature_vectors : np.ndarray, metric = "Euclidean"):
    """
        Perform Nearest Neighbor classification with k=1.

        Args:
            train_feature_vector: Feature vectors of training samples (2D array).
            train_labels: Labels of training samples (1D array).
            test_feature_vectors: Feature vectors of test samples (2D array).
            metric: Distance metric to use ('euclidean', 'manhattan', 'chi_squared', 'mahalanobis').

        Returns:
            Predicted labels for the test samples.
    """

    test_x_len, _ = test_feature_vectors.shape
    train_x_len, _ = train_feature_vector.shape

    y_predict = np.zeros(test_x_len, dtype=int)

    if metric == "Euclidean":
        
        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            euclidean_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = euclidean_distance(x_test, train_feature_vector[j])

                euclidean_distances[j] = distance

            inds = euclidean_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours


    
    elif metric == "Manhattan":
        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            manhattan_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = manhattan_distance(x_test, train_feature_vector[j])

                manhattan_distances[j] = distance

            inds = manhattan_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours

    elif metric == "Chi-Squared":
         for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            chi_squared_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = chi_squared_distance(x_test, train_feature_vector[j])

                chi_squared_distances[j] = distance

            inds = chi_squared_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]


            y_predict[i] = neighbours

    elif metric == "Mahalanobis":
        
        inv_cov_matrix = compute_inverse_covariance_matrix(train_feature_vector)

        for i in range(test_x_len):

            x_test = test_feature_vectors[i]

            neighbours = np.zeros(1)

            mahalanobis_distances = np.zeros(train_x_len, np.float32)

            for j in range(train_x_len):
                
                distance = mahalanobis_distance(x_test, train_feature_vector[j], inv_cov_matrix)

                mahalanobis_distances[j] = distance

            inds = mahalanobis_distances.argsort() 
          
            Y_train_sorted = train_labels[inds] 

            neighbours = Y_train_sorted[0]

            y_predict[i] = neighbours
    
    else:
        raise Exception(f"{metric} distance metric is not supported by this implementation")
    
    return y_predict

In [18]:
def import_labels():

    # labels

    train_labels = []

    with open('imageset/000/train.txt') as f:
        labels = f.readlines()
        for line in labels[1:]:
            _, label = line.split()
            label = label.strip()
            train_labels.append(int(label))

    train_labels = np.array(train_labels)


    test0_labels = []

    with open('test1.txt') as f:
        labels = f.readlines()
        for line in labels[1:]:
            _, label = line.split()
            label = label.strip()
            test0_labels.append(int(label))

    test0_labels = np.array(test0_labels)


    test1_labels = []

    with open('test2.txt') as f:
        labels = f.readlines()
        for line in labels[1:]:
            _, label = line.split()
            label = label.strip()
            test1_labels.append(int(label))

    test1_labels = np.array(test1_labels)

    #print(len(train_labels), len(test0_labels), len(test1_labels))

    return train_labels, test0_labels, test1_labels

In [19]:
train_labels, test0_labels, test1_labels = import_labels()

# Implementation

## Evaluation Function

In [20]:
from sklearn.metrics import accuracy_score

def evaluate_classification(true_labels, predicted_labels):
    """
    Evaluate classification accuracy.
    
    Args:
        true_labels: Ground truth labels.
        predicted_labels: Predicted labels.

    Returns:
        Accuracy score.
    """
    return accuracy_score(true_labels, predicted_labels)

## All metrics with all features

In [80]:
def calculate_results(train, train_labels, test_0_dict, test_0_labels, test_1_dict, test_1_labels):

    distance_metrics = ["Euclidean", "Manhattan", "Chi-Squared", "Mahalanobis"]
    feature_types = ["LBP", "GLCM", "FFT", "Gabor"]

    results = pd.DataFrame(columns=["Method", "Eucledian", "Manhattan", "Chi-Squared", "Mahalanobis"])

    for feature in feature_types:
        
        acc = []

        for metric in distance_metrics:
            test0_predict = nearest_neighbour(train[feature], train_labels, test_0_dict[feature], metric=metric)
            test1_predict = nearest_neighbour(train[feature], train_labels, test_1_dict[feature], metric=metric)

            accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
            accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

            acc.append((accuracy_test0 + accuracy_test1)/2)

        results.loc[len(results)] = [feature, acc[0], acc[1], acc[2], acc[3]]

    return results



## Results with initial feature extractors

In [81]:
initial_results = calculate_results(initial_train, train_labels, initial_test_0, test0_labels, initial_test_1, test1_labels)
initial_results

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.748032,0.762269,0.785532,0.816551
1,GLCM,0.484606,0.496181,0.469444,0.554977
2,FFT,0.631134,0.631019,0.598727,0.510764
3,Gabor,0.292130,0.292361,0.292014,0.271991


## Combinations

In [88]:
def calculate_combinations_result(train, train_labels, test_0_dict, test_0_labels, test_1_dict, test_1_labels):

    from sklearn.preprocessing import StandardScaler

    distance_metrics = ["Euclidean", "Manhattan", "Chi-Squared", "Mahalanobis"]
    feature_types = ["LBP", "GLCM", "FFT", "Gabor"]

    combination_results = pd.DataFrame(columns=['Method', 'Eucledian', 'Manhattan', 'Chi-Squared', 'Mahalanobis'])

    for i in range(len(feature_types)):
        feature1 = feature_types[i]
        for j in range(i+1, len(feature_types)):
            feature2 = feature_types[j]
            combined_train_features = np.concatenate((train[feature1], train[feature2]), axis=1)
            combined_test0_features = np.concatenate((test_0_dict[feature1], test_0_dict[feature2]), axis=1)
            combined_test1_features = np.concatenate((test_1_dict[feature1], test_1_dict[feature2]), axis=1)

            scalar = StandardScaler()
            scalar.fit(combined_train_features)
            combined_train_features = scalar.transform(combined_train_features)
            combined_test0_features = scalar.transform(combined_test0_features)
            combined_test1_features = scalar.transform(combined_test1_features)

            acc = []

            for metric in distance_metrics:
                test0_predict = nearest_neighbour(combined_train_features, train_labels, combined_test0_features, metric=metric)
                test1_predict = nearest_neighbour(combined_train_features, train_labels, combined_test1_features, metric=metric)

                accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
                accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

                acc.append((accuracy_test0 + accuracy_test1)/2)

            combination_results.loc[len(combination_results)] = [f"{feature1} - {feature2}", acc[0], acc[1], acc[2], acc[3]]

    
    for i in range(len(feature_types)):
        feature1 = feature_types[i]
        for j in range(i+1, len(feature_types)):
            feature2 = feature_types[j]
            for k in range(j+1, len(feature_types)):
                feature3 = feature_types[k]
                combined_train_features = np.concatenate((train[feature1], train[feature2], train[feature3]), axis=1)
                combined_test0_features = np.concatenate((test_0_dict[feature1], test_0_dict[feature2], test_0_dict[feature3]), axis=1)
                combined_test1_features = np.concatenate((test_1_dict[feature1], test_1_dict[feature2], test_1_dict[feature3]), axis=1)

                scalar = StandardScaler()
                scalar.fit(combined_train_features)
                combined_train_features = scalar.transform(combined_train_features)
                combined_test0_features = scalar.transform(combined_test0_features)
                combined_test1_features = scalar.transform(combined_test1_features)

                acc = []

                for metric in distance_metrics:

                    test0_predict = nearest_neighbour(combined_train_features, train_labels, combined_test0_features, metric=metric)
                    test1_predict = nearest_neighbour(combined_train_features, train_labels, combined_test1_features, metric=metric)

                    accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
                    accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

                    acc.append((accuracy_test0 + accuracy_test1)/2)

                combination_results.loc[len(combination_results)] = [f"{feature1} - {feature2} - {feature3}", acc[0], acc[1], acc[2], acc[3]]

    combined_train_features = np.concatenate((train["LBP"], train["GLCM"], train["FFT"], train["Gabor"]), axis=1)
    combined_test0_features = np.concatenate((test_0_dict["LBP"], test_0_dict["GLCM"], test_0_dict["FFT"], test_0_dict["Gabor"]), axis=1)
    combined_test1_features = np.concatenate((test_1_dict["LBP"], test_1_dict["GLCM"], test_1_dict["FFT"], test_1_dict["Gabor"]), axis=1)

    scalar = StandardScaler()
    scalar.fit(combined_train_features)
    combined_train_features = scalar.transform(combined_train_features)
    combined_test0_features = scalar.transform(combined_test0_features)
    combined_test1_features = scalar.transform(combined_test1_features)

    acc = []

    for metric in distance_metrics:
        test0_predict = nearest_neighbour(combined_train_features, train_labels, combined_test0_features, metric=metric)
        test1_predict = nearest_neighbour(combined_train_features, train_labels, combined_test1_features, metric=metric)

        accuracy_test0 = evaluate_classification(test0_predict, test_0_labels)
        accuracy_test1 = evaluate_classification(test1_predict, test_1_labels)

        acc.append((accuracy_test0 + accuracy_test1)/2)

    combination_results.loc[len(combination_results)] = ["All Features", acc[0], acc[1], acc[2], acc[3]]

    return combination_results 

In [89]:
initial_combination_results = calculate_combinations_result(initial_train, train_labels, initial_test_0, test0_labels, initial_test_1, test1_labels)
initial_combination_results


,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP - GLCM,0.777778,0.733681,0.008565,0.548264
1,LBP - FFT,0.763542,0.789236,0.010069,0.716088
2,LBP - Gabor,0.696644,0.729630,0.012153,0.591319
3,GLCM - FFT,0.703125,0.692245,0.011806,0.558796
4,GLCM - Gabor,0.601968,0.614583,0.011806,0.494560
5,FFT - Gabor,0.513079,0.534144,0.015162,0.447338
6,LBP - GLCM - FFT,0.787616,0.776157,0.010301,0.568287
7,LBP - GLCM - Gabor,0.747106,0.729282,0.009375,0.533796
8,LBP - FFT - Gabor,0.728125,0.759722,0.011690,0.610417
9,GLCM - FFT - Gabor,0.675463,0.689468,0.012384,0.518519


In [90]:
def create_table(results, combination_results):
    table = pd.concat([results, combination_results])
    return table

initial_table = create_table(initial_results, initial_combination_results)
initial_table

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.748032,0.762269,0.785532,0.816551
1,GLCM,0.484606,0.496181,0.469444,0.554977
2,FFT,0.631134,0.631019,0.598727,0.510764
3,Gabor,0.292130,0.292361,0.292014,0.271991
0,LBP - GLCM,0.777778,0.733681,0.008565,0.548264
1,LBP - FFT,0.763542,0.789236,0.010069,0.716088
2,LBP - Gabor,0.696644,0.729630,0.012153,0.591319
3,GLCM - FFT,0.703125,0.692245,0.011806,0.558796
4,GLCM - Gabor,0.601968,0.614583,0.011806,0.494560
5,FFT - Gabor,0.513079,0.534144,0.015162,0.447338


In [33]:
import cv2
import numpy as np

def gabor_larger_image_bank(image, ksize_list=[7, 11, 15, 21], sigma=5.0, lambd=5.0, gamma=0.5, psi=0, orientations=None):
    """
    Extract flattened Gabor features from an image using multiple kernel sizes, orientations, sigma, and wavelengths.

    Args:
        image (np.ndarray): Grayscale input image.
        ksize_list (list): List of kernel sizes for the Gabor filter.
        sigma_ (list): List of standard deviations of the Gaussian envelope.
        lambd_ (list): List of wavelengths of the sinusoidal factor.
        gamma (float): Aspect ratio.
        psi (float): Phase offset.
        orientations (list): List of orientations in radians. Defaults to [0, π/4, π/2, 3π/4].

    Returns:
        gabor_features (np.ndarray): Flattened feature vector (mean and std for each configuration).
        filtered_images (list): List of filtered images for each configuration.
        kernels (list): List of Gabor filter kernels for each configuration.
    """
    if orientations is None:
        orientations = [0, np.pi / 8, np.pi / 4, 3 * np.pi / 8, np.pi / 2, 
                        5 * np.pi / 8, 3 * np.pi / 4, 7 * np.pi / 8]  # Default orientations

    # Lists to store results
    filtered_images = []
    kernels = []
    gabor_features = []

    # Iterate through all combinations of kernel sizes, sigma, lambd, and orientations
    for ksize in ksize_list:
        for theta in orientations:
            # Create Gabor kernel
            kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
            
            # Apply Gabor filter to the image
            filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
            
            # Calculate mean and standard deviation as features
            mean = filtered.mean()
            std = filtered.std()
            gabor_features.append(mean)
            gabor_features.append(std)
            
            # Store the filtered image and kernel
            filtered_images.append(filtered)
            kernels.append(kernel)

    # Flatten the feature vector
    gabor_features = np.array(gabor_features)

    return gabor_features, filtered_images, kernels


In [ ]:
new_gabor_train = []
new_gabor_test0 = []
new_gabor_test1 = []

new_glcm_train = []
new_glcm_test0 = []
new_glcm_test1 = []

distances = [1, 2, 3, 5]  # Fine and coarse distances
angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]


for image in train_0:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_train.append(flattened_features)
    _, image_features = compute_glcm_features(image, distances, angles)
    new_glcm_train.append(image_features)
    

for image in test_0:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_test0.append(flattened_features)
    _, image_features = compute_glcm_features(image, distances, angles)
    new_glcm_test0.append(image_features)

for image in test_1:
    flattened_features, _, _ = gabor_larger_image_bank(image)
    new_gabor_test1.append(flattened_features)
    _, image_features = compute_glcm_features(image, distances, angles)
    new_glcm_test1.append(image_features)

new_gabor_train = np.array(new_gabor_train)
new_gabor_test0 = np.array(new_gabor_test0)
new_gabor_test1 = np.array(new_gabor_test1)

new_glcm_train = np.array(new_glcm_train)
new_glcm_test0 = np.array(new_glcm_test0)
new_glcm_test1 = np.array(new_glcm_test1)

train_new_gabor_glcm = initial_train.copy()
train_new_gabor_glcm["Gabor"] = new_gabor_train
train_new_gabor_glcm["GLCM"] = new_glcm_train

test0_new_gabor_glcm = initial_test_0.copy()
test0_new_gabor_glcm["Gabor"] = new_gabor_test0
test0_new_gabor_glcm["GLCM"] = new_glcm_test0

test1_new_gabor_glcm = initial_test_1.copy()
test1_new_gabor_glcm["Gabor"] = new_gabor_test1
test1_new_gabor_glcm["GLCM"] = new_glcm_test1



In [38]:
new_fourier_train = []
new_fourier_test0 = []
new_fourier_test1 = []

for image in train_0:
    flattened_features = compute_fft_features(image, 8, 12)
    new_fourier_train.append(flattened_features)

for image in test_0:
    flattened_features = compute_fft_features(image, 8, 12)
    new_fourier_test0.append(flattened_features)

for image in test_1:
    flattened_features = compute_fft_features(image, 8, 12)
    new_fourier_test1.append(flattened_features)

new_fourier_train = np.array(new_fourier_train)
new_fourier_test0 = np.array(new_fourier_test0)
new_fourier_test1 = np.array(new_fourier_test1)

test1_new_gabor_glcm["FFT"] = new_fourier_test1
test0_new_gabor_glcm["FFT"] = new_fourier_test0
train_new_gabor_glcm["FFT"] = new_fourier_train

In [92]:
new_gabor_results = calculate_results(train_new_gabor_glcm, train_labels, test0_new_gabor_glcm, test0_labels, test1_new_gabor_glcm, test1_labels)
new_gabor_results

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.748032,0.762269,0.785532,0.816551
1,GLCM,0.512384,0.523958,0.447917,0.504282
2,FFT,0.683102,0.685185,0.631829,0.483218
3,Gabor,0.413079,0.430324,0.415509,0.396875


In [93]:
new_gabor_results_combined = calculate_combinations_result(train_new_gabor_glcm, train_labels, test0_new_gabor_glcm, test0_labels, test1_new_gabor_glcm, test1_labels)
new_gabor_results_combined

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP - GLCM,0.714120,0.699769,0.009259,0.512847
1,LBP - FFT,0.728356,0.771181,0.011227,0.673958
2,LBP - Gabor,0.558681,0.573032,0.022222,0.439699
3,GLCM - FFT,0.685301,0.687616,0.011690,0.512037
4,GLCM - Gabor,0.624190,0.646528,0.013657,0.449884
5,FFT - Gabor,0.523148,0.527778,0.020949,0.428472
6,LBP - GLCM - FFT,0.732176,0.729745,0.010417,0.508565
7,LBP - GLCM - Gabor,0.661458,0.688079,0.012384,0.441782
8,LBP - FFT - Gabor,0.586227,0.633681,0.019097,0.441204
9,GLCM - FFT - Gabor,0.635185,0.672917,0.014005,0.449884


In [94]:
new_gabor_glcm_fft_table = create_table(new_gabor_results, new_gabor_results_combined)
new_gabor_glcm_fft_table

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.748032,0.762269,0.785532,0.816551
1,GLCM,0.512384,0.523958,0.447917,0.504282
2,FFT,0.683102,0.685185,0.631829,0.483218
3,Gabor,0.413079,0.430324,0.415509,0.396875
0,LBP - GLCM,0.714120,0.699769,0.009259,0.512847
1,LBP - FFT,0.728356,0.771181,0.011227,0.673958
2,LBP - Gabor,0.558681,0.573032,0.022222,0.439699
3,GLCM - FFT,0.685301,0.687616,0.011690,0.512037
4,GLCM - Gabor,0.624190,0.646528,0.013657,0.449884
5,FFT - Gabor,0.523148,0.527778,0.020949,0.428472


# Counter Measures Against Illumination Variations

In [41]:
def normalize_equalize(image):
    """
    Normalize and equalize the input image.

    Args:
        image (np.ndarray): Grayscale input image.

    Returns:
        Normalized and equalized image.
    """
    # Normalize the image
    normalized = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Equalize the histogram
    equalized = cv2.equalizeHist(normalized)
    
    return equalized

norm_train = []

norm_test_0 = []

norm_test_1 = []

for image in train_0:
    norm_train.append(normalize_equalize(image))

for image in test_0:
    norm_test_0.append(normalize_equalize(image))

for image in test_1:
    norm_test_1.append(normalize_equalize(image))

norm_train = np.array(norm_train)
norm_test_0 = np.array(norm_test_0)
norm_test_1 = np.array(norm_test_1)

print(norm_train.shape, norm_test_0.shape, norm_test_1.shape)


(480, 128, 128) (4320, 128, 128) (4320, 128, 128)


In [42]:
def compute_vectors(train, test_0, test_1):
    # Initialize dictionaries to store feature vectors for training and test sets
    train_vector = {}
    test_0_vector = {}
    test_1_vector = {}

    # Define distances and angles for GLCM computation
    distances = [1, 2, 3, 5]  # Fine and coarse distances
    angles = [0, np.pi/8, np.pi/4, 3*np.pi/8, np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]

    # Initialize lists to store feature vectors
    lbp_ = []
    glcm_ = []
    fft_ = []
    gabor_ = []

    # Compute features for training images
    for image in train:
        lbp_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm_.append(image_features)  # Compute GLCM features
        fft_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the training dictionary
    train_vector["LBP"] = np.array(lbp_)
    train_vector["GLCM"] = np.array(glcm_)
    train_vector["FFT"] = np.array(fft_)
    train_vector["Gabor"] = np.array(gabor_)

    # Reset feature lists for test_0
    lbp0_ = []
    glcm0_ = []
    fft0_ = []
    gabor0_ = []

    # Compute features for test_0 images
    for i, image in enumerate(test_0):
        lbp0_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm0_.append(image_features)  # Compute GLCM features
        fft0_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor0_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_0 dictionary
    test_0_vector["LBP"] = np.array(lbp0_)
    test_0_vector["GLCM"] = np.array(glcm0_)
    test_0_vector["FFT"] = np.array(fft0_)
    test_0_vector["Gabor"] = np.array(gabor0_)

    # Reset feature lists for test_1
    lbp1_ = []
    glcm1_ = []
    fft1_ = []
    gabor1_ = []

    # Compute features for test_1 images
    for image in test_1:
        lbp1_.append(compute_lbp(image))  # Compute LBP features
        _, image_features = compute_glcm_features(image, distances, angles)
        glcm1_.append(image_features)  # Compute GLCM features
        fft1_.append(compute_fft_features(image, 8, 12))  # Compute FFT features
        flattened_features, _, _ = gabor_larger_image_bank(image)  # Compute Gabor features
        gabor1_.append(flattened_features)  # Compute Gabor features

    # Store computed features in the test_1 dictionary
    test_1_vector["LBP"] = np.array(lbp1_)
    test_1_vector["GLCM"] = np.array(glcm1_)
    test_1_vector["FFT"] = np.array(fft1_)
    test_1_vector["Gabor"] = np.array(gabor1_)

    return train_vector, test_0_vector, test_1_vector


In [43]:
train_vector, test_0_vector, test_1_vector = compute_vectors(norm_train, norm_test_0, norm_test_1)

In [95]:
normalized_results = calculate_results(train_vector, train_labels, test_0_vector, test0_labels, test_1_vector, test1_labels)

normalized_results 

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.752894,0.762847,0.785532,0.801505
1,GLCM,0.485764,0.488773,0.490509,0.507870
2,FFT,0.682986,0.670139,0.626505,0.536111
3,Gabor,0.429745,0.444213,0.430208,0.374537


In [96]:
normalized_combination_results = calculate_combinations_result(train_vector, train_labels, test_0_vector, test0_labels, test_1_vector, test1_labels)
normalized_combination_results

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP - GLCM,0.676505,0.638310,0.010069,0.520255
1,LBP - FFT,0.732986,0.779977,0.012384,0.691088
2,LBP - Gabor,0.592824,0.589236,0.015741,0.454977
3,GLCM - FFT,0.636343,0.615394,0.011806,0.508218
4,GLCM - Gabor,0.608102,0.607407,0.011458,0.456481
5,FFT - Gabor,0.542824,0.550000,0.015972,0.422801
6,LBP - GLCM - FFT,0.697917,0.672338,0.011806,0.504167
7,LBP - GLCM - Gabor,0.657407,0.648727,0.011343,0.451389
8,LBP - FFT - Gabor,0.627199,0.639236,0.015741,0.462616
9,GLCM - FFT - Gabor,0.629398,0.628819,0.012037,0.453125


In [97]:
normalized_table = create_table(normalized_results, normalized_combination_results)
normalized_table

,Method,Eucledian,Manhattan,Chi-Squared,Mahalanobis
0,LBP,0.752894,0.762847,0.785532,0.801505
1,GLCM,0.485764,0.488773,0.490509,0.507870
2,FFT,0.682986,0.670139,0.626505,0.536111
3,Gabor,0.429745,0.444213,0.430208,0.374537
0,LBP - GLCM,0.676505,0.638310,0.010069,0.520255
1,LBP - FFT,0.732986,0.779977,0.012384,0.691088
2,LBP - Gabor,0.592824,0.589236,0.015741,0.454977
3,GLCM - FFT,0.636343,0.615394,0.011806,0.508218
4,GLCM - Gabor,0.608102,0.607407,0.011458,0.456481
5,FFT - Gabor,0.542824,0.550000,0.015972,0.422801
